In [1]:
import os
!export CUDA_VISIBLE_DEVICES=2
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
%load_ext autoreload
%autoreload 2

import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List
from utils import *

import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm
import torchvision.models as models

from torchprofile import profile_macs
# from torch.nn.parallel import DistributedDataParallel as DDP
# import torch.distributed as dist
from torch.nn.parallel import DataParallel


assert torch.cuda.is_available(), \
"CUDA support is not available."

In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
# set device 
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#set global device
# torch.cuda.set_device(device)
# dist.init_process_group(backend='nccl')

In [4]:
dataloader = get_dataloader("imagenet", batch_size=256)


In [5]:
base_model = models.vgg16(pretrained=True).cuda()


/home/shariff/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shariff/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
evaluate_model(base_model, dataloader, count_nonzero_only=False)


model has test accuracy=68.74%
model has size=527.79 MiB
model has macs=0.44 Gmacs
average inference time is 0.0015 seconds
model has 138.36 M parameters


In [7]:
base_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
temp_model = copy.deepcopy(base_model)

In [9]:
# for relu in list(np.arange(0.1, 1.01, 0.1)):
for relu in [0.3, 0.6, 0.9, 1]:
    compressed_model_1 = copy.deepcopy(temp_model)
    compressed_model_1.classifier._modules[str(4)] = nn.LeakyReLU(negative_slope=nn.Parameter(torch.tensor(relu), requires_grad=False), inplace=True)
    evaluate_model(compressed_model_1, dataloader, count_nonzero_only=False)
    test_accuracy, best_sparse_model_checkpoint = finetune(compressed_model_1, dataloader, num_finetune_epochs=5)
    compressed_model_1.load_state_dict(best_sparse_model_checkpoint['state_dict'])
    temp_model = copy.deepcopy(compressed_model_1)
    print("At relu slope:", relu, ":", test_accuracy)



/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::scalarimplicit". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::leaky_relu_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=64.82%
model has size=527.79 MiB
model has macs=0.44 Gmacs


train:   0%|          | 0/501 [00:00<?, ?it/s]

average inference time is 0.0015 seconds
model has 138.36 M parameters
Finetuning changed Model


train:   0%|          | 0/501 [00:00<?, ?it/s]          

...

train:   0%|          | 0/501 [00:00<?, ?it/s]          

    Epoch 1 test Accuracy 65.44% / Best test Accuracy: 65.44%


train:   0%|          | 0/501 [00:00<?, ?it/s]          

    Epoch 2 test Accuracy 64.86% / Best test Accuracy: 65.44%


train:  46%|████▌     | 229/501 [03:44<04:26,  1.02it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
# evaluate_model(compressed_model_1, dataloader, count_nonzero_only=False)
# list(np.arange(0.1, 1.01, 0.1))

In [11]:
# test_accuracy, best_sparse_model_checkpoint = finetune(compressed_model_1, dataloader)

In [12]:
# compressed_model_1.load_state_dict(best_sparse_model_checkpoint['state_dict'])

In [13]:
# evaluate_model(compressed_model_1, dataloader, count_nonzero_only=False)


In [14]:
# test_accuracy, best_sparse_model_checkpoint = finetune(compressed_model_1, dataloader)

In [15]:
@torch.inference_mode()
def evaluate_topk(
    model: nn.Module,
    dataloader: DataLoader, 
    verbose=True,
    device=None,
    topk = (1,)
) -> float:
  # model.eval()
    mega_list = []
    for inputs, targets in tqdm(dataloader, desc="eval", leave=False, 
                                disable=not verbose):
        # Move the data from CPU to GPU
        if device is None:
            inputs = inputs.cuda()
            targets = targets.cuda()
        else:
            inputs = inputs.to(device)
            targets = targets.to(device)

        # Inference
        outputs = model(inputs)
        maxk = max(topk)
        batch_size = targets.size(0)

        _, y_pred = outputs.topk(k=maxk, dim=1) 
        y_pred = y_pred.t()  

        target_reshaped = targets.view(1, -1).expand_as(y_pred)  
        correct = (y_pred == target_reshaped)  

        list_topk_accs = []  # idx is topk1, topk2, ... etc
        for k in topk:
            ind_which_topk_matched_truth = correct[:k]  # [maxk, B] -> [k, B]
            flattened_indicator_which_topk_matched_truth = ind_which_topk_matched_truth.reshape(-1).float()  # [k, B] -> [kB]
            tot_correct_topk = flattened_indicator_which_topk_matched_truth.float().sum(dim=0, keepdim=True)  # [kB] -> [1]
            topk_acc = tot_correct_topk / batch_size  # topk accuracy for entire batch
            list_topk_accs.append(topk_acc)
        mega_list.append(list_topk_accs)
    return mega_list  # list of topk accuracies for entire batch [topk1, topk2, ... etc]


In [16]:
mega_list = evaluate_topk(base_model, dataloader['val'], topk=(5,))

In [17]:
mega_list = [i[0].item() for i in mega_list]
print(sum(mega_list)/len(mega_list))

0.8989315271377564


In [18]:
mega_list = evaluate_topk(compressed_model_1, dataloader['val'], topk=(5,))
mega_list = [i[0].item() for i in mega_list]
print(sum(mega_list)/len(mega_list))

0.868359375


In [19]:
compressed_model_1

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [20]:
collapse_layers_imagenet_vgg16(compressed_model_1, 4)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [21]:
evaluate_model(compressed_model_1, dataloader, count_nonzero_only=False)

model has test accuracy=66.62%
model has size=463.78 MiB
model has macs=0.42 Gmacs
average inference time is 0.0015 seconds
model has 121.58 M parameters


In [22]:
temp_model = copy.deepcopy(compressed_model_1)
for relu in [0.3, 0.6, 0.9, 1]:
    compressed_model_2 = copy.deepcopy(temp_model)
    compressed_model_2.classifier._modules[str(1)] = nn.LeakyReLU(negative_slope=nn.Parameter(torch.tensor(relu), requires_grad=False), inplace=True)
    evaluate_model(compressed_model_2, dataloader, count_nonzero_only=False)
    test_accuracy, best_sparse_model_checkpoint = finetune(compressed_model_2, dataloader)
    compressed_model_2.load_state_dict(best_sparse_model_checkpoint['state_dict'])
    temp_model = copy.deepcopy(compressed_model_2)
    print("At relu slope:", relu, ":", test_accuracy)



/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::scalarimplicit". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::leaky_relu_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=50.38%
model has size=463.78 MiB
model has macs=0.42 Gmacs


train:   0%|          | 0/501 [00:00<?, ?it/s]

average inference time is 0.0014 seconds
model has 121.58 M parameters
Finetuning changed Model


train:   0%|          | 0/501 [00:00<?, ?it/s]          

...

/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::scalarimplicit". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::leaky_relu_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=61.16%
model has size=463.78 MiB
model has macs=0.42 Gmacs


train:   0%|          | 0/501 [00:00<?, ?it/s]

average inference time is 0.0014 seconds
model has 121.58 M parameters
Finetuning changed Model


train:  64%|██████▍   | 321/501 [05:12<02:55,  1.03it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
evaluate_model(compressed_model_2, dataloader, count_nonzero_only=False)

/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::scalarimplicit". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::leaky_relu_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=62.92%
model has size=463.78 MiB
model has macs=0.42 Gmacs
average inference time is 0.0015 seconds
model has 121.58 M parameters


In [24]:
mega_list = evaluate_topk(compressed_model_2, dataloader['val'], topk=(5,))
mega_list = [i[0].item() for i in mega_list]
print(sum(mega_list)/len(mega_list))

0.8566865801811219


In [25]:
compressed_model_2

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [26]:
w0 = compressed_model_2.classifier._modules[str(0)].weight.data
w6 = compressed_model_2.classifier._modules[str(6)].weight.data
b0 = compressed_model_2.classifier._modules[str(0)].bias.data
b6 = compressed_model_2.classifier._modules[str(6)].bias.data
# model.classifier._modules[str(relu_id + 2)].weight.data = torch.matmul(w3, w0)
compressed_model_2.classifier._modules[str(6)].weight.data = torch.matmul(w6, w0)
# model.classifier._modules[str(relu_id + 2)].bias.data = torch.matmul(w3, b0) + b3
compressed_model_2.classifier._modules[str(6)].bias.data = torch.matmul(w6, b0) + b6

# model.classifier._modules[str(relu_id - 1)] = nn.Identity()
# model.classifier._modules[str(relu_id)] = nn.Identity()
# model.classifier._modules[str(relu_id + 1)] = nn.Identity()
compressed_model_2.classifier._modules[str(0)] = nn.Identity()
compressed_model_2.classifier._modules[str(1)] = nn.Identity()
compressed_model_2.classifier._modules[str(2)] = nn.Identity()

In [27]:
compressed_model_2

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [28]:
mega_list = evaluate_topk(compressed_model_2, dataloader['val'], topk=(5,))
mega_list = [i[0].item() for i in mega_list]
print(sum(mega_list)/len(mega_list))

0.8566865801811219


In [29]:
evaluate_model(compressed_model_2, dataloader, count_nonzero_only=False)

model has test accuracy=63.60%
model has size=151.84 MiB
model has macs=0.34 Gmacs
average inference time is 0.0013 seconds
model has 39.80 M parameters


In [31]:
torch.save(compressed_model_2.state_dict(), "layer_collapse_030_5epochs.pt")